In [1]:
from enum import Enum

class Move(Enum):
    SPLIT = 0
    STEAL = 1

class Result(Enum):
    SPLIT_WON = 0
    SPLIT_LOST = 1
    STOLE_WON = 2
    STOLE_LOST = 3

In [2]:
from abc import ABC

# Abstract class for a Player
class Player(ABC):
    
    def __init__(self):
        super().__init__()
        # Initial
        self.round = 1
        self.money = 0
        # Save history
        self.previous_actions = []
        self.previous_results = []
        self.previous_earnings = []
        # Save state of opponent
        self.opponent = None
    
    def split(self):
        return Move.SPLIT
        
    def steal(self):
        return Move.STEAL
    
    def update(self, choice, result, money):
        self.round += 1
        self.money += money
        self.previous_actions.append(choice)
        self.previous_results.append(result)
        self.previous_earnings.append(money)
    
    def observe(self, opponent):
        self.opponent = opponent
    
    def reset(self):
        self.round = 1
        self.money = 0
        self.previous_actions = []
        self.previous_results = []
        self.previous_earnings = []
        self.opponent = None

In [3]:
import random

# Has equal chance of choosing SHARE/STEAL
class RandomPlayer(Player):
    
    def __init__(self):
        super().__init__()
    
    def action(self):
        if random.uniform(0,1) > 0.5:
            return self.split()
        else:
            return self.steal()


# Always split
class NicePlayer(Player):
    
    def __init__(self):
        super().__init__()
    
    def action(self):
        return self.split()


# Always steal
class GreedyPlayer(Player):
    
    def __init__(self):
        super().__init__()
    
    def action(self):
        return self.steal()


# Alternate between split and steal
# Initial choice is random
class FicklePlayer(Player):
    
    def __init__(self):
        super().__init__()
    
    def action(self):
        if self.round == 1:
            if random.uniform(0,1) > 0.5:
                return self.split()
            else:
                return self.steal()
        else:
            if self.previous_actions[-1] == Move.SPLIT:
                return self.split()
            else:
                return self.share()


# Always split, but once opponent steals, always steal
class CautiousPlayer(Player):
    
    def __init__(self):
        super().__init__()

    def action(self):
        if Result.SPLIT_LOST in self.previous_results:
            return self.steal()
        else:
            return self.split()

In [4]:
def simulate_game(player1, player2, pot=1000, pool=100, max_round=10, print_rounds=False):
    player1.reset()
    player2.reset()
    rnd = 1
    while pot > 0 and rnd <= max_round:
        if print_rounds:
            print("Round", rnd)
        deduct = stimulate_round(player1, player2, pool, print_rounds)
        pot -= deduct
        rnd += 1
    print("Player 1 has", player1.money, "money")
    print("Player 2 has", player2.money, "money")
    print("Remaining money left in pot:", pot)


def stimulate_round(player1, player2, pool, print_rounds):
    action1 = player1.action()
    action2 = player2.action()
    if print_rounds:
        print("Player 1 chooses", action1)
        print("Player 2 chooses", action2)
    if action1 == action2:
        # Both share
        if action1 == Move.SPLIT:
            gains = int(pool / 2)
            player1.update(action1, Result.SPLIT_WON, gains)
            player2.update(action2, Result.SPLIT_WON, gains)
            if print_rounds:
                print("Player 1 gains", gains, "money")
                print("Player 2 gains", gains, "money\n")
        # Both steal
        else:
            player1.update(action1, Result.STOLE_LOST, 0)
            player2.update(action2, Result.STOLE_LOST, 0)
            if print_rounds:
                print("No one gets anything\n")
            pool = 0
    else:
        # Player 1 steal
        if action1 == Move.STEAL:
            player1.update(action1, Result.STOLE_WON, pool)
            player2.update(action2, Result.SPLIT_LOST, 0)
            if print_rounds:
                print("Player 1 gains", pool, "money\n")
        # Player 2 steal
        else:
            player1.update(action1, Result.SPLIT_LOST, 0)
            player2.update(action2, Result.STOLE_WON, pool)
            if print_rounds:
                print("Player 2 gains", pool, "money\n")
    
    player1.observe(player2)
    player2.observe(player1)
    
    return pool

In [5]:
simulate_game(RandomPlayer(), RandomPlayer())

Player 1 has 500 money
Player 2 has 200 money
Remaining money left in pot: 300


In [6]:
simulate_game(NicePlayer(), RandomPlayer())

Player 1 has 200 money
Player 2 has 800 money
Remaining money left in pot: 0


In [7]:
simulate_game(GreedyPlayer(), RandomPlayer())

Player 1 has 500 money
Player 2 has 0 money
Remaining money left in pot: 500


In [8]:
simulate_game(FicklePlayer(), RandomPlayer(), print_rounds=True)

Round 1
Player 1 chooses Move.SPLIT
Player 2 chooses Move.SPLIT
Player 1 gains 50 money
Player 2 gains 50 money

Round 2
Player 1 chooses Move.SPLIT
Player 2 chooses Move.STEAL
Player 2 gains 100 money

Round 3
Player 1 chooses Move.SPLIT
Player 2 chooses Move.SPLIT
Player 1 gains 50 money
Player 2 gains 50 money

Round 4
Player 1 chooses Move.SPLIT
Player 2 chooses Move.STEAL
Player 2 gains 100 money

Round 5
Player 1 chooses Move.SPLIT
Player 2 chooses Move.SPLIT
Player 1 gains 50 money
Player 2 gains 50 money

Round 6
Player 1 chooses Move.SPLIT
Player 2 chooses Move.STEAL
Player 2 gains 100 money

Round 7
Player 1 chooses Move.SPLIT
Player 2 chooses Move.SPLIT
Player 1 gains 50 money
Player 2 gains 50 money

Round 8
Player 1 chooses Move.SPLIT
Player 2 chooses Move.SPLIT
Player 1 gains 50 money
Player 2 gains 50 money

Round 9
Player 1 chooses Move.SPLIT
Player 2 chooses Move.SPLIT
Player 1 gains 50 money
Player 2 gains 50 money

Round 10
Player 1 chooses Move.SPLIT
Player 2 choose

In [9]:
simulate_game(CautiousPlayer(), RandomPlayer(), print_rounds=True)

Round 1
Player 1 chooses Move.SPLIT
Player 2 chooses Move.STEAL
Player 2 gains 100 money

Round 2
Player 1 chooses Move.STEAL
Player 2 chooses Move.SPLIT
Player 1 gains 100 money

Round 3
Player 1 chooses Move.STEAL
Player 2 chooses Move.STEAL
No one gets anything

Round 4
Player 1 chooses Move.STEAL
Player 2 chooses Move.STEAL
No one gets anything

Round 5
Player 1 chooses Move.STEAL
Player 2 chooses Move.SPLIT
Player 1 gains 100 money

Round 6
Player 1 chooses Move.STEAL
Player 2 chooses Move.SPLIT
Player 1 gains 100 money

Round 7
Player 1 chooses Move.STEAL
Player 2 chooses Move.SPLIT
Player 1 gains 100 money

Round 8
Player 1 chooses Move.STEAL
Player 2 chooses Move.SPLIT
Player 1 gains 100 money

Round 9
Player 1 chooses Move.STEAL
Player 2 chooses Move.STEAL
No one gets anything

Round 10
Player 1 chooses Move.STEAL
Player 2 chooses Move.STEAL
No one gets anything

Player 1 has 500 money
Player 2 has 100 money
Remaining money left in pot: 400


In [10]:
simulate_game(CautiousPlayer(), NicePlayer(), print_rounds=True)

Round 1
Player 1 chooses Move.SPLIT
Player 2 chooses Move.SPLIT
Player 1 gains 50 money
Player 2 gains 50 money

Round 2
Player 1 chooses Move.SPLIT
Player 2 chooses Move.SPLIT
Player 1 gains 50 money
Player 2 gains 50 money

Round 3
Player 1 chooses Move.SPLIT
Player 2 chooses Move.SPLIT
Player 1 gains 50 money
Player 2 gains 50 money

Round 4
Player 1 chooses Move.SPLIT
Player 2 chooses Move.SPLIT
Player 1 gains 50 money
Player 2 gains 50 money

Round 5
Player 1 chooses Move.SPLIT
Player 2 chooses Move.SPLIT
Player 1 gains 50 money
Player 2 gains 50 money

Round 6
Player 1 chooses Move.SPLIT
Player 2 chooses Move.SPLIT
Player 1 gains 50 money
Player 2 gains 50 money

Round 7
Player 1 chooses Move.SPLIT
Player 2 chooses Move.SPLIT
Player 1 gains 50 money
Player 2 gains 50 money

Round 8
Player 1 chooses Move.SPLIT
Player 2 chooses Move.SPLIT
Player 1 gains 50 money
Player 2 gains 50 money

Round 9
Player 1 chooses Move.SPLIT
Player 2 chooses Move.SPLIT
Player 1 gains 50 money
Player 2

In [11]:
simulate_game(CautiousPlayer(), GreedyPlayer(), print_rounds=True)

Round 1
Player 1 chooses Move.SPLIT
Player 2 chooses Move.STEAL
Player 2 gains 100 money

Round 2
Player 1 chooses Move.STEAL
Player 2 chooses Move.STEAL
No one gets anything

Round 3
Player 1 chooses Move.STEAL
Player 2 chooses Move.STEAL
No one gets anything

Round 4
Player 1 chooses Move.STEAL
Player 2 chooses Move.STEAL
No one gets anything

Round 5
Player 1 chooses Move.STEAL
Player 2 chooses Move.STEAL
No one gets anything

Round 6
Player 1 chooses Move.STEAL
Player 2 chooses Move.STEAL
No one gets anything

Round 7
Player 1 chooses Move.STEAL
Player 2 chooses Move.STEAL
No one gets anything

Round 8
Player 1 chooses Move.STEAL
Player 2 chooses Move.STEAL
No one gets anything

Round 9
Player 1 chooses Move.STEAL
Player 2 chooses Move.STEAL
No one gets anything

Round 10
Player 1 chooses Move.STEAL
Player 2 chooses Move.STEAL
No one gets anything

Player 1 has 0 money
Player 2 has 100 money
Remaining money left in pot: 900
